## Transfer Learning : Fine Tuning 


In [1]:
# Initiating GPU
import tensorflow as tf
import keras
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("CUDA Support:", tf.test.is_built_with_cuda())
print("GPU Available:", tf.config.list_physical_devices('GPU'))


2025-02-10 22:12:57.302121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739221977.319837  214105 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739221977.325037  214105 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 22:12:57.343229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


CUDA Support: True
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
## Use for the helper script
from helper import *
import datetime
from helper import create_tensorboard_callback

import os

In [3]:
# Getting the data 10% 
walk_through_dir('10_food_classes_10_percent/10_food_classes_10_percent')


There are 2 directories and 0 images in '10_food_classes_10_percent/10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/10_food_classes_10_percent/test/steak'.
There are

In [4]:
# creating directories

train_dir = '10_food_classes_10_percent/10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/10_food_classes_10_percent/test'

- Using models pretrained from tf.keras.applications
---
Not using the image Generator for preprocessing td.data.Dataset


In [5]:
import tensorflow as tf

IMG_SIZE = (224 , 224)
BATCH_SIZE = 32

train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size = IMG_SIZE,
                                                                            label_mode= "categorical",
                                                                            batch_size = BATCH_SIZE,
                                                                        )


test_data = tf.keras.preprocessing.image_dataset_from_directory(directory= test_dir,
                                                                image_size = IMG_SIZE,
                                                                label_mode = "categorical",
                                                                 )


Found 750 files belonging to 10 classes.


I0000 00:00:1739221980.078528  214105 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3931 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Found 2500 files belonging to 10 classes.


In [6]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [7]:
# Example of batch of data 

for images, labels in train_data_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[7.87244892e+00 2.87244892e+00 6.87244892e+00]
   [6.59693909e+00 4.59693909e+00 7.59693909e+00]
   [6.00000000e+00 4.00000000e+00 7.00000000e+00]
   ...
   [5.57654476e+00 9.79080868e+00 1.12193365e+01]
   [1.17857275e+01 7.64284420e+00 8.57140255e+00]
   [1.50714283e+01 6.35714293e+00 4.71428585e+00]]

  [[8.00000000e+00 4.00000000e+00 5.00000000e+00]
   [6.99489784e+00 4.99489784e+00 5.99489784e+00]
   [6.72959185e+00 4.72959185e+00 5.72959185e+00]
   ...
   [4.96939087e+00 5.08673286e+00 7.07142830e+00]
   [6.21940136e+00 6.00000000e+00 5.92854500e+00]
   [7.23979759e+00 5.97448730e+00 4.07142830e+00]]

  [[7.00000000e+00 5.21428585e+00 5.57142830e+00]
   [7.92857170e+00 6.14285755e+00 6.50000000e+00]
   [6.00000000e+00 6.00000000e+00 5.57142830e+00]
   ...
   [1.28060265e+01 4.18881416e+00 6.61734200e+00]
   [5.88263178e+00 6.05613279e+00 5.98469114e+00]
   [2.21425056e+00 7.35717773e+00 5.78571415e+00]]

  ...

  [[1.55362244e+02 1.38147980e+02 1.18147972e+02]
   [1

2025-02-10 22:13:01.778830: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Sequential APIs Vs Functional APIs

- Building Another Feature Extraction model with "Keras Functional API" : Gives more Flexibility with models
- Inputs to the models are to be the same as the model was trained on : See Documentation for this 
---
### Average pool the outputs of the base model (aggregates all the most important weights : Reducing the number of computations)
---
*EfficientNet has Rescaling built in*

In [8]:
# Model 0 
import tf_keras
# Creating base model , with keras.application

base_model = tf_keras.applications.EfficientNetB0(include_top = False)


# Freezing the base model (underlying Weights are not changed)

base_model.trainable = False 

# Create inputs in the mdoel

inputs = tf_keras.layers.Input(shape = (224, 224 , 3), name = "input_layer")

# IF ResNet then the inputs need to be normalized 
# x = tf_keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# Pass the inputs to the base model 

x = base_model(inputs)
print(f"Shape after passing inputs to base_model :{x.shape}")

# Average pool the outputs of the base model (aggregates all the most important weights : Reducing the number of computations)

x = tf_keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer") (x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

# Output Activation Layer 

outputs = tf_keras.layers.Dense(10, activation= "softmax", name = "output_layer")(x)

# Combining inputs and outputs in a model 

model_0 = tf_keras.Model(inputs, outputs)





Shape after passing inputs to base_model :(None, 7, 7, 1280)
Shape after GlobalAveragePooling2D: (None, 1280)


In [9]:
# Compiling the model 

model_0.compile(loss = 'categorical_crossentropy',
                optimizer = tf_keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [10]:
# Fitting 

from tf_keras.callbacks import TensorBoard

history_0 = model_0.fit(train_data_10_percent,
                             epochs=10,
                             steps_per_epoch=len(train_data_10_percent),
                             validation_data=test_data,
                             validation_steps=int(0.25 * len(test_data)),
                             callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                    experiment_name="10_percent_feature_extraction")])


Saving TensorBoard log files to: tensorflow_hub/10_percent_feature_extraction/20250210-221305
Epoch 1/10


I0000 00:00:1739221990.644140  214299 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1739221992.184421  214297 service.cc:148] XLA service 0x7dfccc9dca70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739221992.184588  214297 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2025-02-10 22:13:12.196498: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1739221992.357667  214297 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/24 [==============================] - 11s 169ms/step - loss: 1.8571 - accuracy: 0.4200 - val_loss: 1.2795 - val_accuracy: 0.7615
Epoch 2/10
24/24 [==============================] - 3s 114ms/step - loss: 1.0859 - accuracy: 0.7667 - val_loss: 0.8490 - val_accuracy: 0.8306
Epoch 3/10
24/24 [==============================] - 3s 112ms/step - loss: 0.7917 - accuracy: 0.8320 - val_loss: 0.6863 - val_accuracy: 0.8586
Epoch 4/10
24/24 [==============================] - 3s 112ms/step - loss: 0.6347 - accuracy: 0.8587 - val_loss: 0.5888 - val_accuracy: 0.8668
Epoch 5/10
24/24 [==============================] - 3s 107ms/step - loss: 0.5481 - accuracy: 0.8813 - val_loss: 0.5417 - val_accuracy: 0.8766
Epoch 6/10
24/24 [==============================] - 2s 105ms/step - loss: 0.4722 - accuracy: 0.9027 - val_loss: 0.4867 - val_accuracy: 0.8865
Epoch 7/10
24/24 [==============================] - 3s 107ms/step - loss: 0.4209 - accuracy: 0.9227 - val_loss: 0.4683 - val_accuracy: 0.8882
Epoch 8/10
24/24

In [11]:
# Evaluation of Full Dataset

model_0.evaluate(test_data)

79/79 [==============================] - 5s 62ms/step - loss: 0.4400 - accuracy: 0.8780


[0.43997618556022644, 0.878000020980835]

In [12]:
# Checking the layers in the base_model 

for layer_number, layer in enumerate(base_model.layers):
    print(layer_number, layer.name)

0 input_1
1 rescaling
2 normalization
3 tf.math.multiply
4 stem_conv_pad
5 stem_conv
6 stem_bn
7 stem_activation
8 block1a_dwconv
9 block1a_bn
10 block1a_activation
11 block1a_se_squeeze
12 block1a_se_reshape
13 block1a_se_reduce
14 block1a_se_expand
15 block1a_se_excite
16 block1a_project_conv
17 block1a_project_bn
18 block2a_expand_conv
19 block2a_expand_bn
20 block2a_expand_activation
21 block2a_dwconv_pad
22 block2a_dwconv
23 block2a_bn
24 block2a_activation
25 block2a_se_squeeze
26 block2a_se_reshape
27 block2a_se_reduce
28 block2a_se_expand
29 block2a_se_excite
30 block2a_project_conv
31 block2a_project_bn
32 block2b_expand_conv
33 block2b_expand_bn
34 block2b_expand_activation
35 block2b_dwconv
36 block2b_bn
37 block2b_activation
38 block2b_se_squeeze
39 block2b_se_reshape
40 block2b_se_reduce
41 block2b_se_expand
42 block2b_se_excite
43 block2b_project_conv
44 block2b_project_bn
45 block2b_drop
46 block2b_add
47 block3a_expand_conv
48 block3a_expand_bn
49 block3a_expand_activat